# xView Vehicle Detection — Notebook

This notebook trains/evaluates a YOLO detector (Ultralytics + PyTorch) on the provided xView tiles and runs inference on `future_pass_images/`.

Assumptions:
- Dataset is COCO-style in `annotations.json` with categories including `small-vehicle` and `large-vehicle`.
- Images are under `images/` (train/val) and `future_pass_images/` (inference).
- For speed, we use a lightweight model (yolov8n) and minimal epochs by default.

Outputs:
- Evaluation metrics (precision, recall, mAP).
- Per-image counts of small vs large vehicles in `future_pass_images/`.
- A `results.json` artifact under `outputs/`.


In [1]:
# Environment and imports
from pathlib import Path
import json, glob
from typing import Dict, Any, List

import yaml
import numpy as np
from ultralytics import YOLO

REPO_ROOT = Path.cwd().resolve().parents[0] if (Path.cwd().name == 'notebooks') else Path.cwd().resolve()
print('Repo root:', REPO_ROOT)


Repo root: /Users/kristjantabakou/Desktop/assesment2


## Configuration
We read training and dataset configs to align with the repo pipeline.

In [2]:
train_cfg = REPO_ROOT / 'configs' / 'train.yaml'
data_cfg = REPO_ROOT / 'configs' / 'dataset.yaml'
with open(train_cfg) as f: train = yaml.safe_load(f)
with open(data_cfg) as f: data = yaml.safe_load(f)
train, data


({'model': 'yolov8n',
  'imgsz': 512,
  'epochs': 5,
  'batch': 16,
  'patience': 20,
  'lr0': 0.01,
  'weight_decay': 0.0005,
  'augment': True,
  'seed': 42},
 {'path': '.',
  'train': 'images',
  'val': 'images',
  'names': [],
  'annotations': 'annotations.json',
  'splits': {'train': 'splits/train.txt', 'val': 'splits/val.txt'}})

## Load Trained Model (or Train Quickly)
We load the latest `best.pt` from `outputs/runs/`. If none exists, you can trigger a short training run separately via CLI or add code here to train.

In [3]:
# Find latest trained weights
candidates = glob.glob(str(REPO_ROOT / 'outputs' / 'runs' / '**' / 'weights' / 'best.pt'), recursive=True)
assert candidates, 'No trained weights found. Please run training via CLI or add training cell.'
candidates.sort(key=lambda p: Path(p).stat().st_mtime, reverse=True)
model_path = Path(candidates[0])
print('Using model:', model_path)
model = YOLO(str(model_path))


Using model: /Users/kristjantabakou/Desktop/assesment2/outputs/runs/train4/weights/best.pt


## Evaluation on Validation Split
We evaluate the model using Ultralytics' built-in `val()` to obtain overall metrics.

In [4]:
# Build a minimal dataset YAML pointing to split lists and category names
# We'll derive names from model.names for simplicity here.
names = [model.names[i] for i in sorted(model.names.keys())] if isinstance(model.names, dict) else list(model.names)
val_list = REPO_ROOT / data.get('splits', {}).get('val', 'splits/val.txt')
tmp_yaml = REPO_ROOT / 'outputs' / 'tmp_dataset_nb.yaml'
payload = {
    'train': str(val_list.resolve()),
    'val': str(val_list.resolve()),
    'names': names,
    'nc': len(names),
}
tmp_yaml.parent.mkdir(parents=True, exist_ok=True)
tmp_yaml.write_text(yaml.safe_dump(payload))

metrics = model.val(data=str(tmp_yaml))
metrics_dict = metrics.results_dict
metrics_dict


Ultralytics 8.3.199 🚀 Python-3.13.5 torch-2.8.0 CPU (Apple M4 Pro)
Model summary (fused): 72 layers, 3,010,133 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2345.4±126.8 MB/s, size: 1200.1 KB)
val: Scanning /Users/kristjantabakou/Desktop/assesment2/labels.cache... 101 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 101/101 526.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 0.7it/s 10.0s.6ss
                   all        101       4159       0.67     0.0485     0.0373     0.0135
              building         73       2457      0.309      0.511      0.298      0.121
         small-vehicle         60       1345      0.397       0.12       0.16     0.0439
         large-vehicle         37        182          1          0     0.0125    0.00481
           vehicle-lot          6          6          1          0          0          0
shipping-container-lot          4       

{'metrics/precision(B)': 0.6697576282630835,
 'metrics/recall(B)': 0.04853039334023034,
 'metrics/mAP50(B)': 0.03727641065924193,
 'metrics/mAP50-95(B)': 0.013462994559132966,
 'fitness': 0.013462994559132966}

## Inference on `future_pass_images/` and Vehicle Counting
We run the detector on each future image and count `small-vehicle` and `large-vehicle` detections.

In [5]:
import csv
future_dir = REPO_ROOT / 'future_pass_images'
assert future_dir.exists(), 'future_pass_images directory missing'

pred = model.predict(
    source=str(future_dir),
    conf=0.25,
    iou=0.45,
    save=True,
    project=str(REPO_ROOT / 'outputs' / 'vis'),
    name='infer_nb',
    save_txt=False,
    verbose=False,
)

# Map class ids to names
names_map = model.names if isinstance(model.names, dict) else {i:n for i,n in enumerate(model.names)}
results_list = []
for r in pred:
    img_name = Path(r.path).name
    small, large = 0, 0
    for b in (r.boxes or []):
        cls_id = int(b.cls[0].item()) if b.cls is not None else -1
        cname = names_map.get(cls_id, str(cls_id))
        if cname in ('small-vehicle', 'small_vehicle', 'smallvehicle'):
            small += 1
        elif cname in ('large-vehicle', 'large_vehicle', 'largevehicle'):
            large += 1
    results_list.append({
        'image_id': img_name,
        'small_vehicle_count': int(small),
        'large_vehicle_count': int(large),
    })

len(results_list), results_list[:3]


Results saved to /Users/kristjantabakou/Desktop/assesment2/outputs/vis/infer_nb


(20,
 [{'image_id': 'xView_1132_1.tif',
   'small_vehicle_count': 3,
   'large_vehicle_count': 0},
  {'image_id': 'xView_1132_3.tif',
   'small_vehicle_count': 33,
   'large_vehicle_count': 0},
  {'image_id': 'xView_1182_5.tif',
   'small_vehicle_count': 5,
   'large_vehicle_count': 0}])

## Build `results.json`
We assemble the final JSON file with per-image counts and overall metrics from evaluation.

In [6]:
overall = {
    'train': {
        'average_precision': float(metrics_dict.get('metrics/precision(B)', 0.0)),
        'average_recall': float(metrics_dict.get('metrics/recall(B)', 0.0)),
        'average_mAP50': float(metrics_dict.get('metrics/mAP50(B)', 0.0)),
        'average_mAP50_95': float(metrics_dict.get('metrics/mAP50-95(B)', 0.0)),
        # 'average_IoU': Not directly provided by Ultralytics summary; can be computed from per-class if needed.
    }
}
out = {
    'results': results_list,
    'overall_metrics': overall,
}
out_path = REPO_ROOT / 'outputs' / 'results.json'
out_path.parent.mkdir(parents=True, exist_ok=True)
out_path.write_text(json.dumps(out, indent=2))
print('Wrote', out_path)
out_path.read_text()[:500]


Wrote /Users/kristjantabakou/Desktop/assesment2/outputs/results.json


'{\n  "results": [\n    {\n      "image_id": "xView_1132_1.tif",\n      "small_vehicle_count": 3,\n      "large_vehicle_count": 0\n    },\n    {\n      "image_id": "xView_1132_3.tif",\n      "small_vehicle_count": 33,\n      "large_vehicle_count": 0\n    },\n    {\n      "image_id": "xView_1182_5.tif",\n      "small_vehicle_count": 5,\n      "large_vehicle_count": 0\n    },\n    {\n      "image_id": "xView_1268_15.tif",\n      "small_vehicle_count": 0,\n      "large_vehicle_count": 0\n    },\n    {\n      "image_id": "'